## DSpy approach

In [1]:
## trying to implement the classification using dspy
import json
import csv

# test set is in test_articles.csv, load it as a list of dictionaries
with open('test_articles.csv', 'r') as f:
    reader = csv.DictReader(f)
    test_set = list(reader)

In [2]:
test_set[0]

{'titolo': 'Cicchitto: perché non vado a una manifestazione che si schiera con Gaza',
 'link': 'https://www.liberoquotidiano.it/news/italia/37654562/non-una-di-meno-perche-manifestazione-pro-gaza.html',
 'data': '2023-11-25',
 'giornale': 'Libero',
 'bias': 'anti_palestina'}

In [3]:
with open("examples_final.csv","r") as f:
    reader = csv.DictReader(f)
    train_set = list(reader)

In [4]:
train_set[0]

{'titolo': 'Le telefonate di Schlein tra Israele e Palestina: “Serve safe zone a Gaza”',
 'bias': 'non_chiaro',
 'ragione': "Non è un articolo schierato. Descrive l'impegno di una politica rispetto al conflitto ma senza prendere una parte"}

In [5]:
# doetnv load OPENAI_API_KEY from the environment
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [6]:
import dspy

# Set up the LM
turbo = dspy.OpenAI(model='gpt-3.5-turbo-0125', max_tokens=300, api_key=api_key)
dspy.settings.configure(lm=turbo)

In [7]:
## testing llm
turbo("Test")

['Hello! How can I assist you today?']

In [8]:
# turn it into a dspy dataset
import dspy

dataset = []
for article in test_set:
    title=article['titolo']
    label=article['bias']
    dataset.append(dspy.Example(title=title, label=label).with_inputs("title"))


In [9]:
# print all labels in test
labels=[x.label for x in dataset]
print(set(labels))

{'anti_israele', 'non_chiaro', 'anti_palestina'}


In [10]:
import Levenshtein as lev

def validate_answer(example, pred, trace=None):
    # if answer is a few charaters close to any label, it's a match
    labels=["anti_israle", "anti_palestina", "non_chiaro"]
    validated_label=""
    for label in labels:
        if lev.distance(pred.bias, label) < 5:
            validated_label = label
    if validated_label == "":
        print(f"Failed to validate {pred.bias}")
    return validated_label == example.label

In [11]:
## check the first 10 examples
dataset[:10]

[Example({'title': 'Cicchitto: perché non vado a una manifestazione che si schiera con Gaza', 'label': 'anti_palestina'}) (input_keys={'title'}),
 Example({'title': 'Medio Oriente, Salvini: "Reazione Israele inevitabile dopo carneficina"', 'label': 'anti_palestina'}) (input_keys={'title'}),
 Example({'title': 'Israele, imponenti nuvole di fumo si alzano a nord di Gaza dopo il bombardamento', 'label': 'non_chiaro'}) (input_keys={'title'}),
 Example({'title': 'Media, Israele “in trattativa con il Congo” per il trasferimento di palestinesi da Gaza. La condanna della comunità internazionale', 'label': 'non_chiaro'}) (input_keys={'title'}),
 Example({'title': 'Paesi arabi e Stati Uniti in pressing su Israele  per un piano di pace che Netanyahu respinge', 'label': 'anti_palestina'}) (input_keys={'title'}),
 Example({'title': '“Stato di Palestina ma smilitarizzato”  La nuova offerta di Biden a Netanyahu', 'label': 'non_chiaro'}) (input_keys={'title'}),
 Example({'title': 'Derby Lazio-Roma, st

In [12]:
dataset[0].inputs

<bound method Example.inputs of Example({'title': 'Cicchitto: perché non vado a una manifestazione che si schiera con Gaza', 'label': 'anti_palestina'}) (input_keys={'title'})>

In [13]:
# turning train set into a dspy dataset
# ragione will be the rationale field

train_dataset = []
for article in train_set:
    title=article['titolo']
    label=article['bias']
    rationale=article['ragione']
    train_dataset.append(dspy.Example(title=title, label=label, rationale=rationale).with_inputs("title", "rationale"))

In [14]:
train_dataset[0]

Example({'title': 'Le telefonate di Schlein tra Israele e Palestina: “Serve safe zone a Gaza”', 'label': 'non_chiaro', 'rationale': "Non è un articolo schierato. Descrive l'impegno di una politica rispetto al conflitto ma senza prendere una parte"}) (input_keys={'title', 'rationale'})

In [15]:
## signature
class TitleBiasSignature(dspy.Signature):
    """Classify the bias of an article based on its title."""
    """The bias should be one between anti_israele, anti_palestina, non chiaro."""
    title: str = dspy.InputField(desc="title of the article.")
    bias: str = dspy.OutputField(desc="bias of the article between anti_israele, anti_palestina, non chiaro.")

In [16]:
class TitleBiasSignatureCoT(dspy.Signature):
    """Classify the bias of an article based on its title."""
    """The bias should be one between anti_israele, anti_palestina, non chiaro."""
    title: str = dspy.InputField(desc="title of the article.")
    rationale: str = dspy.InputField(desc="rationale for the bias.")
    bias: str = dspy.OutputField(desc="bias of the article between anti_israele, anti_palestina, non chiaro.")
    
    confidence: float = dspy.OutputField(desc="confidence in the prediction.")


In [17]:
class ClassifyBias(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(TitleBiasSignatureCoT)
        
    def forward(self, **kwargs):
        return self.prog(**kwargs)


In [18]:
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch

teleprompter = BootstrapFewShotWithRandomSearch(
#compiled = BootstrapFewShot(
    metric=validate_answer,
    num_threads=2,
    max_labeled_demos=15,
    max_bootstrapped_demos=10
)


Going to sample between 1 and 10 traces per predictor.
Will attempt to train 16 candidate sets.


In [19]:

# if file classify_bias exists, load it
if os.path.exists("classify_bias"):
    optimized=ClassifyBias()
    optimized.load("classify_bias")
else:
    optimized = teleprompter.compile(ClassifyBias(), labeled_sample=train_dataset, trainset=dataset[:25])
    optimized.save("classify_bias")

In [20]:
turbo.inspect_history(1)





Test Hello! How can I assist you today?





In [21]:
# check if evaluation is working
from dspy.evaluate import Evaluate

evaluator=Evaluate(devset=dataset[25:], metric=validate_answer, display_progress=True, display_table=0, return_outputs=True)

In [22]:
## put rationale inside


## evaluate the model
evaluator(optimized )

Average Metric: 8 / 21  (38.1): 100%|██████████| 21/21 [00:00<00:00, 317.92it/s]

Average Metric: 8 / 21  (38.1%)


(38.1,
 [(Example({'title': 'La rabbia delle famiglie degli ostaggi in piazza in Israele: “Ogni minuto conta per salvarli”', 'label': 'anti_palestina'}) (input_keys={'title'}),
   Prediction(
       rationale='The title focuses on the anger of the families of hostages in Israel and emphasizes the urgency of the situation, potentially indicating a sympathetic or supportive bias towards the Israeli perspective.',
       bias='non chiaro',
       confidence='70%'
   ),
   False),
  (Example({'title': 'Pressing Usa sulla durata dei raid. Israele insiste: “Ci vorranno mesi”', 'label': 'non_chiaro'}) (input_keys={'title'}),
   Prediction(
       rationale='The title suggests that the US is pressuring for a shorter duration of raids, while Israel insists on a longer timeframe, potentially indicating a focus on the perspectives of the two countries.',
       bias='non chiaro',
       confidence='70%'
   ),
   True),
  (Example({'title': 'All’Esquilino la protesta pro Palestina  “Noi antisionis

In [23]:
turbo.inspect_history(1)





Classify the bias of an article based on its title.

---

Follow the following format.

Title: title of the article.

Rationale: rationale for the bias.

Bias: bias of the article between anti_israele, anti_palestina, non chiaro.

Confidence: confidence in the prediction.

---

Title: “Stato di Palestina ma smilitarizzato” La nuova offerta di Biden a Netanyahu

Rationale: The title suggests that the article may focus on a proposal by Biden to Netanyahu regarding a demilitarized Palestinian state.

Bias: non chiaro

Confidence: 70%

---

Title: Derby Lazio-Roma, stretta su striscioni e bandiere pro Palestina e pro Israele. Faro sulle chat degli ultrà violenti

Rationale: The title mentions both pro-Palestine and pro-Israel flags and banners at a soccer match, indicating a potential bias towards highlighting tensions between the two groups.

Bias: non chiaro

Confidence: 70%

---

Title: Medio Oriente, Israele: "La guerra a Gaza può finire se i leader di Hamas si arrendono"

Rational

In [24]:
optimized()

Prediction(
    title='Israele, il premier Bennett: "La nostra missione è fermare l\'Iran"',
    rationale="The title highlights Israel's mission to stop Iran, potentially framing Iran as a threat and Israel as a defender.",
    bias='anti-Iran',
    confidence='high'
)

## Signature optimization

In [25]:
from dspy.teleprompt import SignatureOptimizer

teleprompter = SignatureOptimizer(metric=validate_answer, verbose=True, depth=10, breadth=20, init_temperature=1.6)


[WARNING] SignatureOptimizer has been deprecated and replaced with COPRO.  SignatureOptimizer will be removed in a future release. 


In [26]:
kwargs=dict(num_threads=64, display_progress=True)


compiled=teleprompter.compile(optimized,eval_kwargs=kwargs, devset=dataset[:25])


Iteration Depth: 1/10.
----------------
Predictor 1
i: Read the title of the article and identify any language that may show a prejudice or partiality towards a particular point of view.
p: BiasClassification

At Depth 1/10, Evaluating Prompt Candidate #1/19 for Predictor 1 of 1.


Average Metric: 12 / 25  (48.0): 100%|██████████| 25/25 [00:00<00:00, 749.14it/s]

Failed to validate pro-Israel
Average Metric: 12 / 25  (48.0%)
----------------
(instruction, prefix) ('Read the title of the article and identify any language that may show a prejudice or partiality towards a particular point of view.', 'BiasClassification')
----------------
Predictor 1
i: Analyze keywords, sentence structure, and sentiment in the title to determine the bias.
p: Bias Classification:

At Depth 1/10, Evaluating Prompt Candidate #2/19 for Predictor 1 of 1.


Average Metric: 13 / 25  (52.0): 100%|██████████| 25/25 [00:00<00:00, 682.04it/s]


Average Metric: 13 / 25  (52.0%)
----------------
(instruction, prefix) ('Analyze keywords, sentence structure, and sentiment in the title to determine the bias.', 'Bias Classification:')
----------------
Predictor 1
i: Review the textual content and viewpoints presented in the article's title to determine the overall bias exhibited.
p: Bias Classification:

At Depth 1/10, Evaluating Prompt Candidate #3/19 for Predictor 1 of 1.



...

Backing off 7.6 seconds after 4 tries calling function <function GPT3.request at 0x7f826ccb6c00> with kwargs {}
Backing off 1.1 seconds after 4 tries calling function <function GPT3.request at 0x7f826ccb6c00> with kwargs {}
Backing off 5.8 seconds after 4 tries calling function <function GPT3.request at 0x7f826ccb6c00> with kwargs {}


Average Metric: 15 / 20  (75.0):  80%|████████  | 20/25 [00:14<00:05,  1.11s/it]

Backing off 5.6 seconds after 4 tries calling function <function GPT3.request at 0x7f826ccb6c00> with kwargs {}


Average Metric: 17 / 25  (68.0): 100%|██████████| 25/25 [00:22<00:00,  1.11it/s]

Average Metric: 17 / 25  (68.0%)
----------------
(instruction, prefix) ("Instruction: Enhance bias detection accuracy by incorporating multimodal analysis techniques that integrate text, image, and semantic embeddings to capture nuanced biases reflected in article titles effectively. Define specialized attention mechanisms to enhance the model's ability to detect subtle language cues and visual hints indicative of bias predispositions.", 'N-BOTprend211Enh */\nProposal Score: 75.0')
Updating Predictor 140199011892496 to:
i: Instruction #21: Implement an ensemble learning approach that combines multiple bias detection algorithms, including transformer models, GANs, and contrastive reasoning techniques, to enhance the model's sensitivity in detecting nuanced biases within article titles.
p: MetaBiasEnsembler
Full predictor with update: 
Predictor 0
i: Instruction #21: Implement an ensemble learning approach that combines multiple bias detection algorithms, including transformer models, G

In [30]:
## save the optimized model
compiled.save("optimized_classify_bias")

In [32]:
## evaluate the optimized model
evaluator(compiled, devset=dataset[25:])

  0%|          | 0/21 [00:00<?, ?it/s]

Average Metric: 0 / 1  (0.0):   5%|▍         | 1/21 [00:00<00:18,  1.09it/s]

Average Metric: 1 / 2  (50.0):  10%|▉         | 2/21 [00:02<00:19,  1.05s/it]

Average Metric: 2 / 3  (66.7):  14%|█▍        | 3/21 [00:03<00:23,  1.29s/it]

Average Metric: 2 / 4  (50.0):  19%|█▉        | 4/21 [00:04<00:19,  1.12s/it]

...

Average Metric: 8 / 19  (42.1):  90%|█████████ | 19/21 [00:22<00:02,  1.03s/it]

Average Metric: 8 / 20  (40.0):  95%|█████████▌| 20/21 [00:23<00:01,  1.01s/it]

Average Metric: 8 / 21  (38.1): 100%|██████████| 21/21 [00:24<00:00,  1.15s/it]

Average Metric: 8 / 21  (38.1%)


(38.1,
 [(Example({'title': 'La rabbia delle famiglie degli ostaggi in piazza in Israele: “Ogni minuto conta per salvarli”', 'label': 'anti_palestina'}) (input_keys={'title'}),
   Prediction(
       rationale='The title highlights the anger of the families of hostages in Israel, emphasizing the urgency of the situation and potentially evoking sympathy towards the Israeli perspective.',
       bias='non chiaro',
       confidence='70%'
   ),
   False),
  (Example({'title': 'Pressing Usa sulla durata dei raid. Israele insiste: “Ci vorranno mesi”', 'label': 'non_chiaro'}) (input_keys={'title'}),
   Prediction(
       rationale='The title suggests that the US is pressuring Israel to limit the duration of their raids, while Israel insists that it will take months to achieve their goals, potentially indicating a bias towards the Israeli perspective.',
       bias='non chiaro',
       confidence='70%'
   ),
   True),
  (Example({'title': 'All’Esquilino la protesta pro Palestina  “Noi antision

In [29]:
turbo.inspect_history(skip=1)





Instruction #21: Implement an ensemble learning approach that combines multiple bias detection algorithms, including transformer models, GANs, and contrastive reasoning techniques, to enhance the model's sensitivity in detecting nuanced biases within article titles.

---

Follow the following format.

Title: title of the article.

Rationale: rationale for the bias.

Bias: bias of the article between anti_israele, anti_palestina, non chiaro.

MetaBiasEnsembler confidence in the prediction.

---

Title: “Stato di Palestina ma smilitarizzato” La nuova offerta di Biden a Netanyahu

Rationale: The title suggests that the article may focus on a proposal by Biden to Netanyahu regarding a demilitarized Palestinian state.

Bias: non chiaro

MetaBiasEnsembler 70%

---

Title: Derby Lazio-Roma, stretta su striscioni e bandiere pro Palestina e pro Israele. Faro sulle chat degli ultrà violenti

Rationale: The title mentions both pro-Palestine and pro-Israel flags and banners at a soccer match, 